In [1]:
import pandas as pd
from PIL import Image
import numpy as np
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

#Procesado de img (a 16x16 para agilizar el entrenamiento)
def load_and_process_image(image_path):
    image = Image.open(image_path)
    image = image.resize((16, 16)) # Cambia el tamaño de la imagen a 64x64 para obtener más características HOG
    image = image.convert('L') # Convertir a escala de grises ya que HOG se calcula sobre imágenes en escala de grises
    return np.array(image)

#hog
def extract_features(images):
    hog_features = []
    for image in images:
        fd = hog(image, orientations=8, pixels_per_cell=(16,16), cells_per_block=(1, 1))
        hog_features.append(fd)
    return np.array(hog_features)

#Csv
print("Cargando datos de los CSV...")
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')
val_data = pd.read_csv('val_data.csv')

#Carga+procesado+obtencion de etiquetas
print("procesando imágenes...")
X_train = np.array([load_and_process_image(img_path) for img_path in train_data['path']])
y_train = train_data['label']
X_test = np.array([load_and_process_image(img_path) for img_path in test_data['path']])
y_test = test_data['label']
X_val = np.array([load_and_process_image(img_path) for img_path in val_data['path']])
y_val = val_data['label']
print("imágenes procesadas con éxito.")

# características HOG
print("Extrayendo HOG features...")
X_train_hog = extract_features(X_train)
X_test_hog = extract_features(X_test)
X_val_hog = extract_features(X_val)
print("HOG extraídas con éxito.")

#clasificador SVM
clf = svm.SVC()

# Entrenamiento
print("Entrenando el modelo SVM...")
clf.fit(X_train_hog, y_train)
print("Al fin, entrenado.")

# predicciones
print("Realizando predicciones en test_data...")
y_pred_test = clf.predict(X_test_hog)
print("Realizando predicciones en val_data...")
y_pred_val = clf.predict(X_val_hog)

# Evaluacion
print("Evaluando modelo...")
print('\nAccuracy en el test: ',accuracy_score(y_test, y_pred_test))
print('\n',classification_report(y_test, y_pred_test))
print('\nAccuracy en validación: ',accuracy_score(y_val, y_pred_val))
print('\n',classification_report(y_val, y_pred_val))

# Guardar el modelo
from joblib import dump
dump(clf, 'svm_model.joblib') 
print("Modelo guardado con éxito.")




Cargando los datos de los archivos CSV...
Cargando y procesando las imágenes...
Las imágenes se han cargado y procesado con éxito.
Extrayendo características HOG...
Características HOG extraídas con éxito.
Entrenando el modelo SVM...
Modelo SVM entrenado con éxito.
Realizando predicciones en el conjunto de prueba...
Realizando predicciones en el conjunto de validación...
Evaluando el modelo...

Accuracy en el conjunto de prueba:  0.7180484992613411

               precision    recall  f1-score   support

           0       0.72      1.00      0.84     19928
           1       0.00      0.00      0.00      7825

    accuracy                           0.72     27753
   macro avg       0.36      0.50      0.42     27753
weighted avg       0.52      0.72      0.60     27753


Accuracy en el conjunto de validación:  0.718434707408475

               precision    recall  f1-score   support

           0       0.72      1.00      0.84     19938
           1       0.00      0.00      0.00     

C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metr